In [24]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import logging
import requests
import urllib.request
import json
import numpy as np
from tensorflow.keras.models import load_model
from gensim.models import FastText
from konlpy.tag import Okt
import boto3
import sys
import pandas as pd
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import re

MAX_NB_WORDS = 100000
SNU_model = load_model('SNU_LSTM_Model.h5')
kakao_model = load_model('kakao_LSTM_model.h5')
# SNU_model_shuffle = load_model('SNU_LSTM_Model_shuffle.h5')
similar_model = FastText.load('similar_keyword_model')

class MyTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger

    def __call__(self, sent):
        pos = self.tagger.pos(sent)
        clean_words = []  # 정제된 단어 리스트
        for word in pos:
            # word[1]은 품사를 의미하며, 여기서는 조사, 문장기호, 접두사, Foreign('\n'을 빼주기 위함)인 것은 제외시킴.
            if word[1] not in ['Josa', 'Punctuation', 'Suffix', 'Foreign']:
                if len(word[0]) >= 2:  # 한 글자인 단어들도 의미가 없는 경우가 많으므로 일단 제외.
                    #if word[0] not in ['있다', '했다', '한다', '없다', '된다']:
                    clean_words.append(word[0])
        return clean_words


my_Tokenizer = MyTokenizer(Okt())


In [25]:
import pymysql
import sys
import os
import requests
import base64
import json
import logging
import time
import pandas as pd
import csv
import datetime

host = '' # rds endpoint
port = 3306
username = ''
database = ''
password = ''

def connect_RDS(host, port, username, password, database):
    try:
        conn = pymysql.connect(host = host, user = username, password = password, db = database, port = port, use_unicode = True, charset = 'utf8')
        cursor = conn.cursor()
    
    except:
        logging.error('RDS에 연결되지 않았습니다.')
        sys.exit(1)
        
    return conn, cursor

In [26]:
if sys.version_info[0] < 3:
    from io import StringIO # Python 2.x
else:
    from io import StringIO # Python 3.x

# get your credentials from environment variables
aws_id = ''
aws_secret = ''

client = boto3.client('s3', aws_access_key_id=aws_id,
        aws_secret_access_key=aws_secret)

bucket_name = 'snucsv'

object_key = 'SNU_Data_1200.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('cp949')

# load data
train_df = pd.read_csv(StringIO(csv_string))
label_names = ["label"]
y_train = train_df[label_names].values

train_df['doc_len'] = train_df['document'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(train_df['doc_len'].mean() + train_df['doc_len'].std()).astype(int)
train_df

,document,label,doc_len
0,현재 AI기술은 불완전하기 때문에 가짜뉴스 완전히 걸러낼 수 없다,1,9
1,"‘정치 세대교체’ 바람 일으킨 이준석, 대선엔 출마 못한다",1,8
2,액상형 전자담배는 니코틴이 함유되지 않아 담배가 아니다,1,7
3,한국 백신 접종률 세계 100위권 이하다,1,6
4,광역도시 없는 지역권은 실제 수요와 관계없이 획일적으로 국가교통망 정책에서 소외되고 있다,1,11
...,...,...,...
1219,""" ‘당·정·청 전원회의’는 '운동권 용어'다""?",0,5
1220,"""늘어가는 해양사고, 전북 관내 잠수구조인력 전무(全無)""?",0,6
1221,"""청와대 하루 평균 업무추진비, 이명박 768만원, 박근혜 814만원, 문재인 55만원""",0,10
1222,"국민연금공단 기금운용본부가 전주로 이전한 후, 해외투자자가 국민연금을 '패싱'한다",0,8


**단어 벡터화 및 패딩**

In [27]:
raw_docs_train = train_df['document'].tolist()
num_classes = len(label_names)
print(num_classes)
processed_docs_train = []

for doc in tqdm(raw_docs_train):
    tokens = my_Tokenizer(doc)
    processed_docs_train.append(tokens)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(processed_docs_train)
word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
word_index = tokenizer.word_index
print('dictionary size :', len(word_index))

word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_train

  6%|▋         | 78/1224 [00:00<00:01, 779.20it/s]

1


100%|██████████| 1224/1224 [00:04<00:00, 293.14it/s]

dictionary size : 3999


array([[ 503, 1459,  504, ..., 1461, 1462,    8],
       [   0,   72,  761, ...,   51,  360,    5],
       [   0,    0, 1466, ...,  506,  763,   41],
       ...,
       [ 649,  177,  381, ..., 3992,   13, 3993],
       [3994, 3995, 3996, ...,  248, 3998,    5],
       [   0,    0,    0, ..., 1401, 3999,  496]], dtype=int32)

In [28]:
# 카카오 응답 데이터 전처리
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

train_data.drop_duplicates(subset=['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
text = 'do!!! you expect... people~ to~ read~ the FAQ, etc. and actually accept hard~! atheism?@@'
re.sub(r'[^a-zA-Z ]', '', text) #알파벳과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")# 한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how = 'any')

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()
X_train = []
for sentence in train_data['document']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(X_train)

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer2.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

vocab_size = total_cnt - rare_cnt + 1
tokenizer_kakao = Tokenizer(vocab_size) 
tokenizer_kakao.fit_on_texts(X_train)
X_train = tokenizer_kakao.texts_to_sequences(X_train)

max_len = 30

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()
/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == '':


**사용자에게 받은 데이터 전처리 테스트**

In [29]:
response = ['현재 AI기술은 불완전하기 때문에 가짜뉴스 완전히 걸러낼 수 없다', '‘정치 세대교체’ 바람 일으킨 이준석, 대선엔 출마 못한다', '"늘어가는 해양사고, 전북 관내 잠수구조인력 전무(全無)"?']
tokens_response = []

for i in range(len(response)):
    tokens = my_Tokenizer(response[i])
    tokens_response.append(tokens)
print(tokens_response)

word_seq_response = tokenizer.texts_to_sequences(tokens_response)
print(word_seq_response)
word_seq_response = sequence.pad_sequences(word_seq_response, maxlen=max_seq_len)
print(word_seq_response.shape)

score_list = []
for sco in range(len(word_seq_response)):
    word_seq_response_to_score = word_seq_response[sco].reshape(1,max_seq_len)
    print(word_seq_response_to_score)
    score_list.append(float(SNU_model.predict(word_seq_response_to_score)))
print(score_list)

[['현재', 'AI', '기술', '불완전하기', '때문', '가짜', '뉴스', '완전히', '걸러', '없다'], ['정치', '세대', '교체', '바람', '일으킨', '이준석', '대선', '출마', '한다'], ['늘어가는', '해양사고', '전북', '관내', '잠수', '구조', '인력', '전무']]
[[503, 1459, 504, 1460, 22, 71, 104, 1461, 1462, 8], [72, 761, 1463, 1464, 762, 1465, 51, 360, 5], [3988, 3989, 483, 1080, 3990, 226, 1037, 1458]]
(3, 10)
[[ 503 1459  504 1460   22   71  104 1461 1462    8]]
[[   0   72  761 1463 1464  762 1465   51  360    5]]
[[   0    0 3988 3989  483 1080 3990  226 1037 1458]]
[0.5054650902748108, 0.5144935846328735, 0.511650025844574]


**입력 데이터 처리 함수**

In [30]:
# ===========================================================================================================================
# 카카오톡 응답 데이터 score

def kakao_data_score(response):
    objectivity = ''
    response = okt.morphs(response, stem=True) # 토큰화
    response = [word for word in response if not word in stopwords] # 불용어 제거
    encoded = tokenizer_kakao.texts_to_sequences([response]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(kakao_model.predict(pad_new)) # 예측
    if(score > 0.8 or score < 0.2):
        objectivity = '객관성이 매우 낮습니다.'
    elif(score > 0.65 or score < 0.35):
        objectivity = '객관성이 낮습니다.'
    else:
        objectivity = '객관성이 높습니다.'
    
    return objectivity


# ===========================================================================================================================
# 단어 토큰화 및 빈도순 정렬 후 추출

def tfidf_Vectorizer(response_list):
    tfidf_Vectorizer = TfidfVectorizer(tokenizer=my_Tokenizer, min_df=1) # df 값(단어가 몇 문장들에서 등장하였는지)을 최소 'min_df' 값으로 설정.
    X = tfidf_Vectorizer.fit_transform(response_list).toarray()
    print(X.shape)    # X(2차원 배열)의 행,열 수를 출력.
    print(tfidf_Vectorizer.vocabulary_)   # 각 단어들이 배열 X에서 몇번째 열(인덱스 값)에 해당하는지 출력.


    #pandas를 활용하여 각 단어들의 각 문장에서의 tf-idf 값들을 모두 더하고, 내림차순으로 정렬하여 상위 n개 출력
    count = X.sum(axis=0)    # 2차원 배열 X에서 각 열을 기준으로 합을 구함. (각 단어들의 '최종' tf-idf 값으로 간주.)
    word_count = pd.DataFrame({
        '단어' : tfidf_Vectorizer.get_feature_names(),
        '빈도' : count.flat
    })
    sorted_df = word_count.sort_values('빈도', ascending=False)
    print(sorted_df.head(10), "\n")

    word_ = list(np.array(sorted_df['단어'].tolist()))
    return(word_)


# ===========================================================================================================================
# 연관키워드 추출

def similar_keyword(word_):
    similar_list = []

    if len(word_) == 1:
        similar_keywords = similar_model.wv.most_similar(word_[0])

        for i in range(len(similar_keywords)):
            similar_list.append(similar_keywords[i][0])
    return similar_list


# ===========================================================================================================================
# MySQL 데이터베이스에 응답데이터, 시간 저장

def MySQL_data_save(response):
    global cursor, conn
    conn, cursor = connect_RDS(host, port, username, password, database)

    now = datetime.datetime.now()
    Korea_DatetimeNow = now + datetime.timedelta(hours = 9)
    DatetimeNow = Korea_DatetimeNow.strftime('%Y-%m-%d %H:%M:%S')

    try:
        with conn.cursor() as cursor:
            query = "INSERT INTO factcheck.article_data (article_data, date) VALUES(%s, %s)"
            val = (response, DatetimeNow)
            cursor.execute(query, val)
            conn.commit()

    finally:
        conn.close()
        
        
# ===========================================================================================================================
# MySQL 데이터베이스에 키워드 랭킹 반영

def keyword_rank(word_):
    rank_list = []
    conn, cursor = connect_RDS(host, port, username, password, database)

    try:
        with conn.cursor() as cursor:
            cursor.execute('SELECT keyword FROM keyword_rank')
            row = cursor.fetchall()

        for i in range(len(row)):
            rank_list.append(row[i][0])

        with conn.cursor() as cursor:
            for i in range(0,1):
                if word_[i] in rank_list:
                    query = "UPDATE keyword_rank SET count = count + 1 WHERE keyword = (%s)"
                    val = (word_[i])
                    cursor.execute(query, val)
                    conn.commit()
                else:
                    query = "INSERT INTO factcheck.keyword_rank (keyword, count) VALUES(%s, 1)"
                    val = (word_[i])
                    cursor.execute(query, val)
                    conn.commit()

        top10_list = []
        with conn.cursor() as cursor:
            cursor.execute("SELECT keyword FROM keyword_rank ORDER BY count DESC LIMIT 10")
            row_top = cursor.fetchall()
            for i in range(len(row_top)):
                top10_list.append(row_top[i][0])

    finally:
        conn.close()
    return top10_list


# ===========================================================================================================================
# SNU 기사 검색      

def SNU_Search(word_):
    title_list2 = []
    # link_list2 = []
    score_list2 = []

    try:
#             str_expr = "document.str.contains('{}', case=False)".format(word_[0]) and "document.str.contains('{}', case=False)".format(word_[1])
        str_expr = "document.str.contains('{}', case=False)".format(word_[0])

        df_q = train_df.query(str_expr, engine="python")

        title = list(np.array(df_q['document'].tolist()))
        score_SNU = list(np.array(df_q['label'].tolist()))
        # link = list(np.array(df_q['link'].tolist()))

        for i in range(len(title)):
            title_list2.append(title[i])
        #     link_list2.append(link[i])
            score_list2.append(score_SNU[i])

    except:
        pass
    return title_list2, score_list2


# ===========================================================================================================================
# 네이버 뉴스 검색

def Naver_Search(response):
    baseurl = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query="

    url = baseurl + urllib.parse.quote_plus(response)

    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.find_all(class_ = 'news_tit')

    title_list = []
    link_list = []

    for i in title:
        title_list.append(i.attrs['title'])
        link_list.append(i.attrs['href'])

# ================================================================

    tokens_response = []
    score_list = []

    for i in range(len(title_list)):
        tokens = my_Tokenizer(title_list[i])
        tokens_response.append(tokens)

    word_seq_response = tokenizer.texts_to_sequences(tokens_response)
    word_seq_response = sequence.pad_sequences(word_seq_response, maxlen=max_seq_len)

    for sco in range(len(title_list)):
        word_seq_response_to_score = word_seq_response[sco].reshape(1,max_seq_len)
        score_list.append(float(SNU_model.predict(word_seq_response_to_score)))
    
    return title_list, link_list, score_list

**Flask 웹 서버**

In [ ]:
#-*-coding:utf-8-*-

from flask import Flask, request
import json
from urllib import parse

app = Flask(__name__)

@app.route('/', methods = ['POST'])
def Article_keyword():
    if request.method == 'POST':
        params = request.get_data()
        params_decode = str(params.decode('utf-8'))
        urldecode = parse.unquote(params_decode)
        
        # 사용자 요청 데이터
        response = urldecode.split('=')[1]
        response_list = []
        response_list.append(response)
        
        word_ = tfidf_Vectorizer(response_list)
        print('1')
        similar_list = similar_keyword(word_)
        print('2')
        MySQL_data_save(response)
        print('3')
        top10_list = keyword_rank(word_)
        print('4')
        SNU_title, SNU_score = SNU_Search(word_)
        print('5')
        Naver_title, Naver_link, Naver_score = Naver_Search(response)
        print('6')
        if len(similar_list) == 0:
            similar_list.append('연관 검색어가 없습니다.')
        if len(SNU_title) == 0:
            SNU_title.append('관련 뉴스 결과가 없습니다.')
        if len(SNU_score) == 0:
            SNU_score.append('x')
        if len(Naver_title) == 0:
            Naver_title.append('관련 뉴스 결과가 없습니다.')
        if len(Naver_link) == 0:
            Naver_link.append('x')
        if len(Naver_score) == 0:
            Naver_score.append('x')
            
        data_ = {
            'response':response,
            'similar_list':similar_list,
            'SNU_title':SNU_title,
            'SNU_score':SNU_score,
            'Naver_title':Naver_title,
            'Naver_link':Naver_link,
            'Naver_score':Naver_score,
            'top10_list':top10_list
            }
        
        print('입력 데이터 : \n', response)
        print('\n')
        print('연관 검색어 : \n', similar_list)
        print('\n')
        print('네이버 뉴스 제목 : \n', Naver_title)
        print('\n')
        print('네이버 뉴스 링크 : \n', Naver_link)
        print('\n')
        print('네이버 뉴스 score : \n', Naver_score)
        print('\n')
        try:
            print('SNU 기사 제목 : \n', SNU_title)
            print('\n')
    #         print('SNU 기사 링크 : ', link_list2)
            print('SNU 기사 score : \n', SNU_score)
            print('\n')
        except:
            print('SNU 기사가 없습니다. \n')
        print('인기키워드 랭킹 : \n', top10_list)
        print('\n')
        print('\n')
        
        return json.dumps(data_, ensure_ascii=False)

@app.route('/kakao', methods = ['POST'])
def kakao_():
    if request.method == 'POST':
        params = request.get_data()
        params_decode = str(params.decode('utf-8'))
        urldecode = parse.unquote(params_decode)
        
        # 사용자 요청 데이터
        response = urldecode.split('=')[1]
        response_list = []
        response_list.append(response)
        
        objectivity = kakao_data_score(response)
        word_ = tfidf_Vectorizer(response_list)
        
        similar_list = similar_keyword(word_)
        MySQL_data_save(response)
        top10_list = keyword_rank(word_)
        
        SNU_title, SNU_score = SNU_Search(word_)
        Naver_title, Naver_link, Naver_score = Naver_Search(response)
        
        if len(similar_list) == 0:
            similar_list.append('연관 검색어가 없습니다.')
        if len(SNU_title) == 0:
            SNU_title.append('관련 뉴스 결과가 없습니다.')
        if len(SNU_score) == 0:
            SNU_score.append('x')
        if len(Naver_title) == 0:
            Naver_title.append('관련 뉴스 결과가 없습니다.')
        if len(Naver_link) == 0:
            Naver_link.append('x')
        if len(Naver_score) == 0:
            Naver_score.append('x')
        
        data_ = {
            'response':response,
            'objectivity':objectivity,
            'similar_list':similar_list,
            'SNU_title':SNU_title,
            'SNU_score':SNU_score,
            'Naver_title':Naver_title,
            'Naver_link':Naver_link,
            'Naver_score':Naver_score,
            'top10_list':top10_list
            }
        
        print('입력 데이터 : \n', response)
        print('\n')
        print('객관성 level : \n', objectivity)
        print('\n')
        print('연관 검색어 : \n', similar_list)
        print('\n')
        print('네이버 뉴스 제목 : \n', Naver_title)
        print('\n')
        print('네이버 뉴스 링크 : \n', Naver_link)
        print('\n')
        print('네이버 뉴스 score : \n', Naver_score)
        print('\n')
        try:
            print('SNU 기사 제목 : \n', SNU_title)
            print('\n')
    #         print('SNU 기사 링크 : ', link_list2)
            print('SNU 기사 score : \n', SNU_score)
            print('\n')
        except:
            print('SNU 기사가 없습니다. \n')
        print('인기키워드 랭킹 : \n', top10_list)
        print('\n')
        print('\n')
        
        return json.dumps(data_, ensure_ascii=False)
    
    
if __name__ == '__main__':
    app.run(host='0.0.0.0', port = 8080)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)


(1, 2)
{'안녕하세요': 0, '오소리': 1}
      단어        빈도
0  안녕하세요  0.707107
1    오소리  0.707107 

1
2
3
4
5


125.133.174.241 - - [09/Sep/2021 15:24:34] "POST / HTTP/1.1" 200 -


6
입력 데이터 : 
 안녕하세요 오소리에요


연관 검색어 : 
 ['연관 검색어가 없습니다.']


네이버 뉴스 제목 : 
 ['독수리술·오소리술 들어는 보셨습니까? 전설적인 평안도 사냥꾼의 음식', '[어제보다 행복하기]\xa0비움의 행복', '뮤지션 #천용성 의 두 번째 목소리', '주문을 걸어 (원슈타인 화보 & 인터뷰)', "'가상화폐, 문제는 다 文 정부 때문'이라는 언론"]


네이버 뉴스 링크 : 
 ['http://www.nocutnews.co.kr/news/4655459', 'http://www.newscj.com/news/articleView.html?idxno=788375', 'http://www.elle.co.kr/article/56416', 'http://www.wkorea.com/2020/12/24/%ec%a3%bc%eb%ac%b8%ec%9d%84-%ea%b1%b8%ec%96%b4-%ec%9b%90%ec%8a%88%ed%83%80%ec%9d%b8/?utm_source=naver&utm_medium=partnership', 'http://www.nocutnews.co.kr/news/4910068']


네이버 뉴스 score : 
 [0.5058199763298035, 0.5058199763298035, 0.5058199763298035, 0.5060451626777649, 0.5118430256843567]


SNU 기사 제목 : 
 ['관련 뉴스 결과가 없습니다.']


SNU 기사 score : 
 ['x']


인기키워드 랭킹 : 
 ['안녕하세요', '유료', '카카오', '25', '코로나', '123123', '카카오톡', 'nameasdfasdfas', '입니다', 'dcxzvdszvzcd']




(1, 2)
{'안녕하세요': 0, '오소리': 1}
      단어        빈도
0  안녕하세요  0.707107
1    오소리  0.707107 



125.133.174.241 - - [09/Sep/2021 15:24:39] "POST /kakao HTTP/1.1" 200 -


입력 데이터 : 
 안녕하세요 오소리에요


객관성 level : 
 객관성이 높습니다.


연관 검색어 : 
 ['연관 검색어가 없습니다.']


네이버 뉴스 제목 : 
 ['독수리술·오소리술 들어는 보셨습니까? 전설적인 평안도 사냥꾼의 음식', '[어제보다 행복하기]\xa0비움의 행복', '뮤지션 #천용성 의 두 번째 목소리', '주문을 걸어 (원슈타인 화보 & 인터뷰)', "'가상화폐, 문제는 다 文 정부 때문'이라는 언론"]


네이버 뉴스 링크 : 
 ['http://www.nocutnews.co.kr/news/4655459', 'http://www.newscj.com/news/articleView.html?idxno=788375', 'http://www.elle.co.kr/article/56416', 'http://www.wkorea.com/2020/12/24/%ec%a3%bc%eb%ac%b8%ec%9d%84-%ea%b1%b8%ec%96%b4-%ec%9b%90%ec%8a%88%ed%83%80%ec%9d%b8/?utm_source=naver&utm_medium=partnership', 'http://www.nocutnews.co.kr/news/4910068']


네이버 뉴스 score : 
 [0.5058199763298035, 0.5058199763298035, 0.5058199763298035, 0.5060451626777649, 0.5118430256843567]


SNU 기사 제목 : 
 ['관련 뉴스 결과가 없습니다.']


SNU 기사 score : 
 ['x']


인기키워드 랭킹 : 
 ['안녕하세요', '유료', '카카오', '25', '코로나', '123123', '카카오톡', 'nameasdfasdfas', '입니다', 'dcxzvdszvzcd']




